In [1]:
from pytorch_optim_training_manager import train_manager
import torch
import torchvision
import torchvision.transforms as transforms
import models
import os

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [4]:
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)

P1

In [39]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

In [40]:
training_set = torchvision.datasets.FashionMNIST('./data', train=True, transform=transform, download=True)
validation_set = torchvision.datasets.FashionMNIST('./data', train=False, transform=transform, download=True)

In [41]:
training_loader = torch.utils.data.DataLoader(training_set, batch_size=128, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=128, shuffle=False)

In [42]:
model = models.CNN_Simple()

In [10]:
torch.save(model.state_dict(), 'models/CNN_Simple.pt')

In [62]:
loss_fn = torch.nn.CrossEntropyLoss()

start again here if evaluating for multiple optimizers

In [61]:
model.load_state_dict(torch.load('models/CNN_Simple.pt'))

<All keys matched successfully>

In [63]:
optimizer = torch.optim.RMSprop(model.parameters())

In [64]:
manager = train_manager(model, loss_fn, optimizer,training_loader,validation_loader,device=device)

In [65]:
losses = manager.train(20, verbose=True, eval_all_epochs=True, eval_mode='accuracy')

epoch0: train_loss: 0.7445
eval_loss: 0.7654
epoch1: train_loss: 0.4870
eval_loss: 0.8421
epoch2: train_loss: 0.4218
eval_loss: 0.8280
epoch3: train_loss: 0.3985
eval_loss: 0.8498
epoch4: train_loss: 0.3834
eval_loss: 0.8499
epoch5: train_loss: 0.3681
eval_loss: 0.8615
epoch6: train_loss: 0.3534
eval_loss: 0.8421
epoch7: train_loss: 0.3487
eval_loss: 0.8549
epoch8: train_loss: 0.3395
eval_loss: 0.8593
epoch9: train_loss: 0.3377
eval_loss: 0.8275
epoch10: train_loss: 0.3334
eval_loss: 0.8585
epoch11: train_loss: 0.3271
eval_loss: 0.8700
epoch12: train_loss: 0.3228
eval_loss: 0.8732
epoch13: train_loss: 0.3229
eval_loss: 0.8624
epoch14: train_loss: 0.3187
eval_loss: 0.8546
epoch15: train_loss: 0.3144
eval_loss: 0.8699
epoch16: train_loss: 0.3065
eval_loss: 0.8671
epoch17: train_loss: 0.3106
eval_loss: 0.8713
epoch18: train_loss: 0.3165
eval_loss: 0.8644
epoch19: train_loss: 0.2994
eval_loss: 0.8545


In [66]:
print(torch.max(torch.tensor(losses[1])))

tensor(0.8732)


In [67]:
torch.save(torch.tensor(losses[1]), 'results/RMSprop_P1.pt')

P2

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor()])

In [6]:
training_set = torchvision.datasets.MNIST('./data', train=True, transform=transform, download=True)
validation_set = torchvision.datasets.MNIST('./data', train=False, transform=transform, download=True)

In [7]:
training_loader = torch.utils.data.DataLoader(training_set, batch_size=64, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=64, shuffle=False)

In [8]:
model = models.VAE()

In [9]:
torch.save(model.state_dict(), 'models/VAE.pt')

In [62]:
class VAE_loss(torch.nn.Module):
    def __init__(self):
        super(VAE_loss, self).__init__()
        self.loss_fn = torch.nn.CrossEntropyLoss()
    def forward(self, preds, labels):
        x, mean, logvar = preds
        reproduction_loss = self.loss_fn(x, labels)
        # KLD = -0.5 * torch.sum(1 + logvar - mean**2 - logvar.exp())
        # (reproduction_loss + KLD) / len(labels)
        return reproduction_loss

In [63]:
loss_fn = VAE_loss()

restart here for a different optimizer

In [94]:
model.load_state_dict(torch.load('models/VAE.pt'))

<All keys matched successfully>

In [95]:
optimizer = torch.optim.Adam(model.parameters())

In [96]:
manager = train_manager(model, loss_fn, optimizer,training_loader,validation_loader,device=device)

In [97]:
losses = manager.train(20, verbose=True, eval_all_epochs=True, eval_mode='loss')

epoch0: train_loss: 5.7042
eval_loss: 5.6861
epoch1: train_loss: 5.6861
eval_loss: 5.6861
epoch2: train_loss: 5.6861
eval_loss: 5.6861
epoch3: train_loss: 5.6861
eval_loss: 5.6861
epoch4: train_loss: 5.6861
eval_loss: 5.6861
epoch5: train_loss: 5.6861
eval_loss: 5.6861
epoch6: train_loss: 5.6861
eval_loss: 5.6861
epoch7: train_loss: 5.6861
eval_loss: 5.6861
epoch8: train_loss: 5.6861
eval_loss: 5.6861
epoch9: train_loss: 5.6861
eval_loss: 5.6861
epoch10: train_loss: 5.6861
eval_loss: 5.6861
epoch11: train_loss: 5.6861
eval_loss: 5.6861
epoch12: train_loss: 5.6861
eval_loss: 5.6861
epoch13: train_loss: 5.6861
eval_loss: 5.6861
epoch14: train_loss: 5.6861
eval_loss: 5.6861
epoch15: train_loss: 5.6861
eval_loss: 5.6861
epoch16: train_loss: 5.6861
eval_loss: 5.6861
epoch17: train_loss: 5.6861
eval_loss: 5.6861
epoch18: train_loss: 5.6861
eval_loss: 5.6861
epoch19: train_loss: 5.6861
eval_loss: 5.6861


In [98]:
print(torch.min(torch.tensor(losses[1])))

tensor(5.6861)


In [99]:
torch.save(torch.tensor(losses[1]), 'results/AdaGrad_P2.pt')

P3

In [100]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

In [101]:
training_set = torchvision.datasets.FashionMNIST('./data', train=True, transform=transform, download=True)
validation_set = torchvision.datasets.FashionMNIST('./data', train=False, transform=transform, download=True)
training_loader = torch.utils.data.DataLoader(training_set, batch_size=128, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=128, shuffle=False)

In [102]:
model = models.VAE()

In [103]:
loss_fn = VAE_loss()

In [104]:
torch.save(model.state_dict(), 'models/VAE.pt')

restart here if evaluating for multiple optimizers

In [131]:
model.load_state_dict(torch.load('models/VAE.pt'))

<All keys matched successfully>

In [132]:
optimizer = torch.optim.RMSprop(model.parameters())

In [133]:
manager = train_manager(model, loss_fn, optimizer, training_loader, validation_loader, device=device)

In [134]:
losses = manager.train(20, verbose=True, eval_all_epochs=True, eval_mode='loss')

epoch0: train_loss: 5.6933
eval_loss: 5.6861
epoch1: train_loss: 5.6861
eval_loss: 5.6861
epoch2: train_loss: 5.6861
eval_loss: 5.6861
epoch3: train_loss: 5.6861
eval_loss: 5.6861
epoch4: train_loss: 5.8317
eval_loss: 5.9984
epoch5: train_loss: 6.0267
eval_loss: 6.0671
epoch6: train_loss: 6.1017
eval_loss: 6.0813
epoch7: train_loss: 6.0685
eval_loss: 6.1842
epoch8: train_loss: 6.1787
eval_loss: 6.1613
epoch9: train_loss: 6.2528
eval_loss: 6.2936
epoch10: train_loss: 6.4205
eval_loss: 6.5108
epoch11: train_loss: 6.5214
eval_loss: 6.5335
epoch12: train_loss: 6.5845
eval_loss: 6.6530
epoch13: train_loss: 6.6453
eval_loss: 6.5651
epoch14: train_loss: 6.5402
eval_loss: 6.5377
epoch15: train_loss: 6.5074
eval_loss: 6.4810
epoch16: train_loss: 6.4605
eval_loss: 6.4335
epoch17: train_loss: 6.4326
eval_loss: 6.4359
epoch18: train_loss: 6.4298
eval_loss: 6.4354
epoch19: train_loss: 6.4394
eval_loss: 6.4798


In [135]:
print(torch.min(torch.tensor(losses[1])))

tensor(5.6861)


In [136]:
torch.save(torch.tensor(losses[1]), 'results/RMSprop_P3.pt')

P4